# Cyclone Extractor & Excel Export for Tableau

This notebook allows you to:
1. **Select a time period (year)** and **region** to search for cyclones
2. **Extract cyclone track data** from IBTrACS database
3. **Export to Excel** with temporal progression for Tableau analysis

## 📋 Prerequisites
- IBTrACS database will be automatically downloaded if not present
- Openpyxl package for Excel export

## Step 1: Import Libraries and Setup

In [2]:
import sys
from pathlib import Path
import pandas as pd
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import custom modules
from config import PROJECT_ROOT, DATA_DIR, OUTPUT_DIR, RAW_DIR

print("✅ Libraries loaded successfully")
print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📁 Data Directory: {DATA_DIR}")
print(f"📁 Output Directory: {OUTPUT_DIR}")

✅ Libraries loaded successfully
📁 Project Root: /Users/enochtham/Library/CloudStorage/OneDrive-UniversityofOklahoma/VSC - OU/Ookla_final
📁 Data Directory: /Users/enochtham/Library/CloudStorage/OneDrive-UniversityofOklahoma/VSC - OU/Ookla_final/data
📁 Output Directory: /Users/enochtham/Library/CloudStorage/OneDrive-UniversityofOklahoma/VSC - OU/Ookla_final/data/output


## Step 2: Define Your Search Parameters

**Customize these parameters to search for cyclones:**

In [8]:
# ═══════════════════════════════════════════════════════════════
# CUSTOMIZE YOUR SEARCH HERE
# ═══════════════════════════════════════════════════════════════

# Time period
SEARCH_YEAR = 2024
# Years to search for cyclones

# Geographic region (bounding box)
# Format: [min_lon, min_lat, max_lon, max_lat]
REGION_BOUNDS = {
    'name': 'Southeast Asia',
    'bounds': [95, -15, 145, 10],  # Covers Indonesia, Philippines, etc.
    # Alternative regions (uncomment to use):
    #'bounds': [100, -10, 120, 5],   # Indonesia/Java/Sumatra
    # 'bounds': [115, 5, 130, 20],    # Philippines
    # 'bounds': [70, 5, 95, 25],      # Bay of Bengal/India
}

# Minimum intensity filter (optional)
# Set to 0 to include all tropical depressions, or higher to filter
MIN_WIND_SPEED = 34  # knots (34 = Tropical Storm, 64 = Hurricane/Typhoon)

print("🔍 Search Parameters:")
print(f"   Year: {SEARCH_YEAR}")
print(f"   Region: {REGION_BOUNDS['name']}")
print(f"   Bounds: {REGION_BOUNDS['bounds']}")
print(f"   Min Wind Speed: {MIN_WIND_SPEED} knots")

🔍 Search Parameters:
   Year: 2024
   Region: Southeast Asia
   Bounds: [95, -15, 145, 10]
   Min Wind Speed: 34 knots


## Step 3: Download IBTrACS Database (if needed)

In [9]:
def download_ibtracs_database():
    """Download IBTrACS database if not present"""
    
    ibtracs_dir = RAW_DIR / 'ibtracs'
    ibtracs_dir.mkdir(parents=True, exist_ok=True)
    ibtracs_file = ibtracs_dir / 'IBTrACS.since1980.list.v04r01.csv'
    
    if ibtracs_file.exists():
        print(f"✅ IBTrACS database already exists ({ibtracs_file.stat().st_size / 1e6:.1f} MB)")
        return ibtracs_file
    
    print("📥 Downloading IBTrACS database (this may take a few minutes)...")
    
    url = "https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/csv/ibtracs.since1980.list.v04r01.csv"
    
    try:
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        
        with open(ibtracs_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    percent = (downloaded / total_size) * 100
                    print(f"\r   Progress: {percent:.1f}%", end='')
        
        print(f"\n✅ Downloaded IBTrACS database ({ibtracs_file.stat().st_size / 1e6:.1f} MB)")
        return ibtracs_file
        
    except Exception as e:
        print(f"❌ Error downloading IBTrACS: {e}")
        print("   You can manually download from:")
        print("   https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/")
        return None

# Download or check for database
ibtracs_file = download_ibtracs_database()

✅ IBTrACS database already exists (141.3 MB)


## Step 4: Search for Cyclones in Region

In [10]:
def search_cyclones_in_region(year, bounds, min_wind=0):
    """
    Search for cyclones in a specific region and time period
    
    Parameters:
        year: Year to search
        bounds: [min_lon, min_lat, max_lon, max_lat]
        min_wind: Minimum wind speed in knots
    
    Returns:
        Dictionary of cyclones found
    """
    
    print(f"\n🔍 Searching for cyclones in {year}...")
    
    if ibtracs_file is None:
        print("❌ No IBTrACS database available")
        return {}
    
    # Load IBTrACS data
    print("   Loading IBTrACS database...")
    df = pd.read_csv(ibtracs_file, skiprows=[1], low_memory=False)
    
    # Filter by year (check both SEASON and SEASON+1 for late-year cyclones)
    year_data = df[(df['SEASON'] == year) | (df['SEASON'] == year + 1)].copy()
    
    print(f"   Found {len(year_data)} total records in {year}")
    
    # Convert coordinates to numeric
    year_data['LAT'] = pd.to_numeric(year_data['LAT'], errors='coerce')
    year_data['LON'] = pd.to_numeric(year_data['LON'], errors='coerce')
    year_data['WMO_WIND'] = pd.to_numeric(year_data['WMO_WIND'], errors='coerce')
    
    # Filter by region bounds
    min_lon, min_lat, max_lon, max_lat = bounds
    region_data = year_data[
        (year_data['LAT'] >= min_lat) & 
        (year_data['LAT'] <= max_lat) &
        (year_data['LON'] >= min_lon) & 
        (year_data['LON'] <= max_lon)
    ]
    
    # Find unique cyclones in region
    cyclones_found = {}
    
    for name in region_data['NAME'].dropna().unique():
        if name == 'NOT_NAMED':
            continue
            
        cyclone_data = region_data[region_data['NAME'] == name].copy()
        
        # Check if peak intensity meets minimum
        max_wind = cyclone_data['WMO_WIND'].max()
        if pd.notna(max_wind) and max_wind >= min_wind:
            
            # Get cyclone details
            cyclone_data = cyclone_data.sort_values('ISO_TIME')
            
            cyclones_found[name] = {
                'name': name,
                'season': cyclone_data['SEASON'].iloc[0],
                'basin': cyclone_data['BASIN'].iloc[0] if 'BASIN' in cyclone_data.columns else 'Unknown',
                'num_points': len(cyclone_data),
                'max_wind': max_wind,
                'start_date': cyclone_data['ISO_TIME'].iloc[0],
                'end_date': cyclone_data['ISO_TIME'].iloc[-1],
                'data': cyclone_data
            }
    
    # Print summary
    print(f"\n✅ Found {len(cyclones_found)} cyclones in region:")
    print("=" * 70)
    for name, info in cyclones_found.items():
        print(f"   {name:15s} | {info['start_date'][:10]} to {info['end_date'][:10]} | Max: {info['max_wind']:.0f} kts | Points: {info['num_points']}")
    print("=" * 70)
    
    return cyclones_found

# Search for cyclones
cyclones = search_cyclones_in_region(
    year=SEARCH_YEAR,
    bounds=REGION_BOUNDS['bounds'],
    min_wind=MIN_WIND_SPEED
)


🔍 Searching for cyclones in 2024...
   Loading IBTrACS database...
   Found 10466 total records in 2024

✅ Found 7 cyclones in region:
   NEVILLE         | 2024-03-09 to 2024-03-13 | Max: 45 kts | Points: 29
   MEGAN           | 2024-03-13 to 2024-03-17 | Max: 90 kts | Points: 33
   OLGA            | 2024-04-04 to 2024-04-07 | Max: 95 kts | Points: 23
   UNNAMED         | 2024-12-24 to 2025-05-12 | Max: 45 kts | Points: 171
   TALIAH          | 2025-01-31 to 2025-02-07 | Max: 50 kts | Points: 32
   DIANNE          | 2025-03-26 to 2025-03-28 | Max: 40 kts | Points: 13
   ERROL           | 2025-04-09 to 2025-04-17 | Max: 110 kts | Points: 61


## Step 5: Process Cyclone Data for Tableau

Add temporal tracking and categorization for analysis

In [20]:
def categorize_intensity(wind_kts):
    """Categorize cyclone intensity from wind speed (Saffir-Simpson scale)"""
    if pd.isna(wind_kts):
        return 'Unknown'
    elif wind_kts < 34:
        return 'Tropical Depression'
    elif wind_kts < 64:
        return 'Tropical Storm'
    elif wind_kts < 83:
        return 'Category 1 Hurricane'
    elif wind_kts < 96:
        return 'Category 2 Hurricane'
    elif wind_kts < 113:
        return 'Category 3 Hurricane'
    elif wind_kts < 137:
        return 'Category 4 Hurricane'
    else:
        return 'Category 5 Hurricane'

def process_cyclone_for_tableau(cyclone_info):
    """
    Process a single cyclone's data for Tableau visualization
    Adds temporal and analytical fields
    """
    
    df = cyclone_info['data'].copy()
    
    # Convert time to datetime
    df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'], errors='coerce')
    
    # Sort by time
    df = df.sort_values('ISO_TIME').reset_index(drop=True)
    
    # Add temporal fields for animation
    df['Date'] = df['ISO_TIME'].dt.date
    df['DateTime'] = df['ISO_TIME']
    df['Day_Number'] = (df['ISO_TIME'] - df['ISO_TIME'].min()).dt.days + 1
    df['Hour_Number'] = ((df['ISO_TIME'] - df['ISO_TIME'].min()).dt.total_seconds() / 3600).astype(int)
    
    # Add point sequence
    df['Track_Point'] = range(1, len(df) + 1)
    
    # Intensity categorization
    df['Intensity_Category'] = df['WMO_WIND'].apply(categorize_intensity)
    
    # Intensity level for color coding (1-7 scale)
    def intensity_level(wind):
        if pd.isna(wind): return 0
        elif wind < 34: return 1
        elif wind < 64: return 2
        elif wind < 83: return 3
        elif wind < 96: return 4
        elif wind < 113: return 5
        elif wind < 137: return 6
        else: return 7
    
    df['Intensity_Level'] = df['WMO_WIND'].apply(intensity_level)
    
    # Movement speed (km/h) between points
    df['Speed_kmh'] = 0.0
    for i in range(1, len(df)):
        if pd.notna(df.loc[i, 'LAT']) and pd.notna(df.loc[i-1, 'LAT']):
            from math import radians, cos, sin, asin, sqrt
            
            lat1, lon1 = radians(df.loc[i-1, 'LAT']), radians(df.loc[i-1, 'LON'])
            lat2, lon2 = radians(df.loc[i, 'LAT']), radians(df.loc[i, 'LON'])
            
            # Haversine formula
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            c = 2 * asin(sqrt(a))
            km = 6371 * c  # Earth radius in km
            
            time_diff = (df.loc[i, 'ISO_TIME'] - df.loc[i-1, 'ISO_TIME']).total_seconds() / 3600
            if time_diff > 0:
                df.loc[i, 'Speed_kmh'] = km / time_diff
    
    # Metadata
    df['Cyclone_Name'] = cyclone_info['name']
    df['Season'] = cyclone_info['season']
    df['Basin'] = cyclone_info['basin']
    
    # Select and rename columns for Tableau
    columns_for_tableau = [
        'Cyclone_Name',
        'Season',
        'Basin',
        'DateTime',
        'Date',
        'Day_Number',
        'Hour_Number',
        'Track_Point',
        'LAT',
        'LON',
        'WMO_WIND',
        'WMO_PRES',
        'Intensity_Category',
        'Intensity_Level',
        'Speed_kmh',
        'STORM_SPEED',
        'STORM_DIR'
    ]
    
    # Filter to available columns
    columns_for_tableau = [col for col in columns_for_tableau if col in df.columns]
    
    return df[columns_for_tableau]

# Process all cyclones
print("\n🔄 Processing cyclones for Tableau...")
processed_cyclones = {}

for name, info in cyclones.items():
    print(f"   Processing {name}...")
    processed_cyclones[name] = process_cyclone_for_tableau(info)

print(f"✅ Processed {len(processed_cyclones)} cyclones")


🔄 Processing cyclones for Tableau...
   Processing UNNAMED...
   Processing TALIAH...
   Processing DIANNE...
   Processing ERROL...
✅ Processed 4 cyclones


## Step 6: Preview Data

Let's look at a sample of the processed data

In [21]:
# Show sample data from first cyclone
if processed_cyclones:
    first_cyclone = list(processed_cyclones.keys())[0]
    sample_df = processed_cyclones[first_cyclone]
    
    print(f"\n📊 Sample data from {first_cyclone}:")
    print(f"   Total track points: {len(sample_df)}")
    print(f"\n   First 5 points:")
    display(sample_df.head())
    
    print(f"\n   Data summary:")
    print(f"   - Duration: {sample_df['Day_Number'].max()} days")
    print(f"   - Peak intensity: {sample_df['WMO_WIND'].max():.0f} knots")
    print(f"   - Intensity categories:")
    for cat, count in sample_df['Intensity_Category'].value_counts().items():
        print(f"      • {cat}: {count} points")
else:
    print("⚠️  No cyclones found to preview")


📊 Sample data from UNNAMED:
   Total track points: 171

   First 5 points:


,Cyclone_Name,Season,Basin,DateTime,Date,Day_Number,Hour_Number,Track_Point,LAT,LON,WMO_WIND,WMO_PRES,Intensity_Category,Intensity_Level,Speed_kmh,STORM_SPEED,STORM_DIR
0,UNNAMED,2025,SI,2024-12-24 00:00:00,2024-12-24,1,0,1,-12.5,96.7,15.0,1007,Tropical Depression,1,0.000000,4,295
1,UNNAMED,2025,SI,2024-12-24 03:00:00,2024-12-24,1,3,2,-12.4,96.5,NaN,,Unknown,0,8.132436,5,290
2,UNNAMED,2025,SI,2024-12-24 06:00:00,2024-12-24,1,6,3,-12.3,96.2,15.0,1006,Tropical Depression,1,11.477152,8,290
3,UNNAMED,2025,SI,2024-12-24 09:00:00,2024-12-24,1,9,4,-12.1,95.7,NaN,,Unknown,0,19.572101,11,295
4,UNNAMED,2025,SI,2024-12-24 12:00:00,2024-12-24,1,12,5,-11.8,95.2,15.0,1005,Tropical Depression,1,21.269001,12,315



   Data summary:
   - Duration: 140 days
   - Peak intensity: 45 knots
   - Intensity categories:
      • Unknown: 82 points
      • Tropical Depression: 76 points
      • Tropical Storm: 13 points


## Step 7: Export to Excel for Tableau

Export options:
- **Single file**: All cyclones in one Excel file with separate sheets
- **Multiple files**: One Excel file per cyclone

Each Excel file will have temporal tracking so you can animate the cyclone movement in Tableau!

In [22]:
def export_to_excel(processed_cyclones, export_mode='single'):
    """
    Export cyclone data to Excel for Tableau
    
    Parameters:
        processed_cyclones: Dictionary of processed cyclone DataFrames
        export_mode: 'single' = one file with multiple sheets, 
                    'multiple' = one file per cyclone
    
    Returns:
        List of created files
    """
    
    if not processed_cyclones:
        print("❌ No cyclones to export")
        return []
    
    # Create output directory
    output_dir = OUTPUT_DIR / 'cyclone_tracks' / f"{SEARCH_YEAR}_{REGION_BOUNDS['name'].replace(' ', '_')}"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    created_files = []
    
    if export_mode == 'single':
        # Single file with multiple sheets
        filename = f"cyclones_{SEARCH_YEAR}_{REGION_BOUNDS['name'].replace(' ', '_')}.xlsx"
        output_file = output_dir / filename
        
        print(f"\n📊 Exporting to single Excel file...")
        print(f"   File: {filename}")
        
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            
            # Combined sheet with all cyclones
            all_cyclones = pd.concat(processed_cyclones.values(), ignore_index=True)
            all_cyclones.to_excel(writer, sheet_name='All_Cyclones', index=False)
            print(f"   ✓ Sheet 'All_Cyclones': {len(all_cyclones)} rows")
            
            # Individual sheets for each cyclone
            for name, df in processed_cyclones.items():
                # Excel sheet names have max 31 characters
                sheet_name = name[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"   ✓ Sheet '{sheet_name}': {len(df)} rows")
        
        print(f"✅ Exported to: {output_file}")
        created_files.append(output_file)
        
    else:  # Multiple files
        print(f"\n📊 Exporting to multiple Excel files...")
        
        for name, df in processed_cyclones.items():
            filename = f"{name}_{SEARCH_YEAR}.xlsx"
            output_file = output_dir / filename
            
            df.to_excel(output_file, sheet_name='Track', index=False)
            print(f"   ✓ {filename}: {len(df)} rows")
            created_files.append(output_file)
        
        print(f"✅ Exported {len(created_files)} files to: {output_dir}")
    
    return created_files

# Choose export mode
EXPORT_MODE = 'single'  # Change to 'multiple' for separate files

# Export
export_files = export_to_excel(processed_cyclones, export_mode=EXPORT_MODE)

# Show file sizes
if export_files:
    print("\n📁 File Details:")
    for file in export_files:
        size_kb = file.stat().st_size / 1024
        print(f"   {file.name}: {size_kb:.1f} KB")


📊 Exporting to single Excel file...
   File: cyclones_2025_Southeast_Asia.xlsx
   ✓ Sheet 'All_Cyclones': 277 rows
   ✓ Sheet 'UNNAMED': 171 rows
   ✓ Sheet 'TALIAH': 32 rows
   ✓ Sheet 'DIANNE': 13 rows
   ✓ Sheet 'ERROL': 61 rows
✅ Exported to: /Users/enochtham/Library/CloudStorage/OneDrive-UniversityofOklahoma/VSC - OU/Ookla_final/data/output/cyclone_tracks/2025_Southeast_Asia/cyclones_2025_Southeast_Asia.xlsx

📁 File Details:
   cyclones_2025_Southeast_Asia.xlsx: 55.4 KB


In [ ]:
def export_to_excel(processed_cyclones, export_mode='single'):
    """
    Export cyclone data to Excel for Tableau
    
    Parameters:
        processed_cyclones: Dictionary of processed cyclone DataFrames
        export_mode: 'single' = one file with multiple sheets, 
                    'multiple' = one file per cyclone
    
    Returns:
        List of created files
    """
    
    if not processed_cyclones:
        print("❌ No cyclones to export")
        return []
    
    # Create output directory
    output_dir = OUTPUT_DIR / 'cyclone_tracks' / f"{SEARCH_YEAR}_{REGION_BOUNDS['name'].replace(' ', '_')}"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    created_files = []
    
    if export_mode == 'single':
        # Single file with multiple sheets
        filename = f"cyclones_{SEARCH_YEAR}_{REGION_BOUNDS['name'].replace(' ', '_')}.xlsx"
        output_file = output_dir / filename
        
        print(f"\n📊 Exporting to single Excel file...")
        print(f"   File: {filename}")
        
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            
            # Combined sheet with all cyclones
            all_cyclones = pd.concat(processed_cyclones.values(), ignore_index=True)
            all_cyclones.to_excel(writer, sheet_name='All_Cyclones', index=False)
            print(f"   ✓ Sheet 'All_Cyclones': {len(all_cyclones)} rows")
            
            # Individual sheets for each cyclone
            for name, df in processed_cyclones.items():
                # Excel sheet names have max 31 characters
                sheet_name = name[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"   ✓ Sheet '{sheet_name}': {len(df)} rows")
        
        print(f"✅ Exported to: {output_file}")
        created_files.append(output_file)
        
    else:  # Multiple files
        print(f"\n📊 Exporting to multiple Excel files...")
        
        for name, df in processed_cyclones.items():
            filename = f"{name}_{SEARCH_YEAR}.xlsx"
            output_file = output_dir / filename
            
            df.to_excel(output_file, sheet_name='Track', index=False)
            print(f"   ✓ {filename}: {len(df)} rows")
            created_files.append(output_file)
        
        print(f"✅ Exported {len(created_files)} files to: {output_dir}")
    
    return created_files

# Choose export mode
EXPORT_MODE = 'single'  # Change to 'multiple' for separate files

# Export
export_files = export_to_excel(processed_cyclones, export_mode=EXPORT_MODE)

# Show file sizes
if export_files:
    print("\n📁 File Details:")
    for file in export_files:
        size_kb = file.stat().st_size / 1024
        print(f"   {file.name}: {size_kb:.1f} KB")


📊 Exporting to single Excel file...
   File: cyclones_2019_Southeast_Asia.xlsx
   ✓ Sheet 'All_Cyclones': 535 rows
   ✓ Sheet 'OWEN': 11 rows
   ✓ Sheet 'PENNY': 37 rows
   ✓ Sheet 'UNNAMED': 123 rows
   ✓ Sheet 'WUTIP': 2 rows
   ✓ Sheet 'SAVANNAH': 58 rows
   ✓ Sheet 'TREVOR': 33 rows
   ✓ Sheet 'VERONICA': 14 rows
   ✓ Sheet 'WALLACE': 43 rows
   ✓ Sheet 'LILI': 47 rows
   ✓ Sheet 'PHANFONE': 20 rows
   ✓ Sheet 'CLAUDIA': 63 rows
   ✓ Sheet 'FERDINAND': 23 rows
   ✓ Sheet 'KROVANH': 61 rows
✅ Exported to: /Users/enochtham/Library/CloudStorage/OneDrive-UniversityofOklahoma/VSC - OU/Ookla_final/data/output/cyclone_tracks/2019_Southeast_Asia/cyclones_2019_Southeast_Asia.xlsx

📁 File Details:
   cyclones_2019_Southeast_Asia.xlsx: 104.5 KB


## Step 8: Tableau Animation Guide

### How to animate cyclone tracks in Tableau:

1. **Import the Excel file** into Tableau
   - Connect to the Excel file
   - Select the "All_Cyclones" sheet (or individual cyclone sheet)

2. **Create the map visualization:**
   - Drag `LON` to Columns
   - Drag `LAT` to Rows
   - Change both to Dimension (right-click → Dimension)
   - Change mark type to "Line"

3. **Add cyclone paths:**
   - Drag `Cyclone_Name` to Color
   - Drag `Track_Point` to Path
   - This will draw connected lines for each cyclone

4. **Add intensity coloring:**
   - Instead of `Cyclone_Name` on Color, use `Intensity_Level`
   - Change to continuous color scale
   - Use red gradient (low = light, high = dark red)

5. **Animate over time:**
   - Drag `Day_Number` or `DateTime` to Pages shelf
   - Click the play button to animate!
   - Adjust speed with the slider

6. **Add tooltips:**
   - Drag to Tooltip: `Cyclone_Name`, `DateTime`, `WMO_WIND`, `Intensity_Category`
   - Format tooltip to show relevant details

7. **Optional enhancements:**
   - Add `Speed_kmh` to Size for variable track thickness
   - Filter by `Basin` or `Season`
   - Create calculated field for category colors

### Key Fields for Analysis:
- **Temporal**: `DateTime`, `Date`, `Day_Number`, `Hour_Number`
- **Spatial**: `LAT`, `LON`
- **Intensity**: `WMO_WIND`, `Intensity_Category`, `Intensity_Level`
- **Movement**: `Speed_kmh`, `STORM_SPEED`, `STORM_DIR`
- **Identity**: `Cyclone_Name`, `Season`, `Basin`

## Summary Statistics

Quick overview of all extracted cyclones

In [ ]:
if processed_cyclones:
    print("\n" + "=" * 80)
    print(f"CYCLONE EXTRACTION SUMMARY - {SEARCH_YEAR}")
    print("=" * 80)
    print(f"\n📍 Region: {REGION_BOUNDS['name']}")
    print(f"   Bounds: {REGION_BOUNDS['bounds']}")
    print(f"   Min Wind Speed Filter: {MIN_WIND_SPEED} knots")
    
    print(f"\n🌀 Cyclones Found: {len(processed_cyclones)}")
    print("\nDetailed Statistics:")
    print("-" * 80)
    
    summary_data = []
    for name, df in processed_cyclones.items():
        summary_data.append({
            'Cyclone': name,
            'Basin': df['Basin'].iloc[0],
            'Duration (days)': df['Day_Number'].max(),
            'Track Points': len(df),
            'Peak Wind (kts)': df['WMO_WIND'].max(),
            'Peak Category': df.loc[df['WMO_WIND'].idxmax(), 'Intensity_Category'],
            'Start Date': df['Date'].min(),
            'End Date': df['Date'].max()
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('Peak Wind (kts)', ascending=False)
    display(summary_df)
    
    print(f"\n📊 Overall Statistics:")
    print(f"   Total track points: {sum(len(df) for df in processed_cyclones.values())}")
    print(f"   Average duration: {summary_df['Duration (days)'].mean():.1f} days")
    print(f"   Strongest cyclone: {summary_df.iloc[0]['Cyclone']} ({summary_df.iloc[0]['Peak Wind (kts)']:.0f} kts)")
    
    print(f"\n✅ Data exported and ready for Tableau analysis!")
    print("=" * 80)
else:
    print("\n⚠️  No cyclones found matching your criteria.")
    print("   Try adjusting:")
    print("   - Expanding the region bounds")
    print("   - Lowering MIN_WIND_SPEED")
    print("   - Checking a different year")